In [1]:
import numpy as np
import tensorflow as tf
print(tf.VERSION)

1.8.0


In [2]:
def cnn_model(features,labels,mode):
    input_layer = tf.reshape(features['x'],[-1,28,28,1])
    conv_1 = tf.layers.conv2d(inputs=input_layer,filters=32,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    pool_1 = tf.layers.max_pooling2d(inputs=conv_1,pool_size = [2,2],strides = 2)
    conv_2 = tf.layers.conv2d(inputs=pool_1,filters=64,kernel_size=[5,5],padding='same',activation=tf.nn.relu)
    pool_2 = tf.layers.max_pooling2d(inputs=conv_2,pool_size = [2,2],strides = 2)
    flat = tf.reshape(pool_2,[-1,7 * 7 * 64])
    dense = tf.layers.dense(inputs=flat,units=1024,activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense,rate=0.4,training=(mode==tf.estimator.ModeKeys.TRAIN))
    logits = tf.layers.dense(inputs=dropout,units=10)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    predictions = {
        'classes' : tf.argmax(logits,axis=1),
        'probabilities' : tf.nn.softmax(logits)
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    eval_metric_ops = {
        'accuracy' : tf.metrics.accuracy(labels=labels,predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=eval_metric_ops)

In [3]:
mnist = tf.contrib.learn.datasets.load_dataset('mnist')
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels,dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels,dtype=np.int32)
classifier = tf.estimator.Estimator(model_fn=cnn_model)


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpwg0zemkq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoint

In [4]:
train_input = tf.estimator.inputs.numpy_input_fn(x={'x' : train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
classifier.train(input_fn=train_input,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpwg0zemkq/model.ckpt.
INFO:tensorflow:loss = 2.3173664, step = 0
INFO:tensorflow:global_step/sec: 82.4324
INFO:tensorflow:loss = 0.12256774, step = 100 (1.214 sec)
INFO:tensorflow:global_step/sec: 86.8842
INFO:tensorflow:loss = 0.117238015, step = 200 (1.151 sec)
INFO:tensorflow:global_step/sec: 87.0519
INFO:tensorflow:loss = 0.12399943, step = 300 (1.149 sec)
INFO:tensorflow:global_step/sec: 87.4306
INFO:tensorflow:loss = 0.048690736, step = 400 (1.144 sec)
INFO:tensorflow:global_step/sec: 87.1351
INFO:tensorflow:loss = 0.148526, step = 500 (1.148 sec)
INFO:tensorflow:global_step/sec: 83.7709
INFO:tensorflow:loss = 0.15605517, step = 600 (1.194 sec)
INFO:tensorflow:global_step/sec: 84.7743
INFO:tenso

In [5]:
eval_input = tf.estimator.inputs.numpy_input_fn(x={'x' : eval_data},y=eval_labels,num_epochs=1,shuffle=False)
eval_results = classifier.evaluate(input_fn=eval_input)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-21-11:06:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpwg0zemkq/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-21-11:06:40
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9812, global_step = 10000, loss = 0.076786935
{'accuracy': 0.9812, 'loss': 0.076786935, 'global_step': 10000}
